# 加速：非同步爬蟲

* 了解非同步爬蟲加速原理與實作

## 作業目標

* 比較一下非同步爬蟲跟多線程爬蟲的差異是什麼？各自的優缺點為何？

'''
Your Input
'''
非同步爬蟲
優點：
1.程式執行當下（不用等到執行完畢）就繼續往下執行，共用session,共用連續開很多個瀏覽器分頁，有效的利用系統資源

2.所謂的非同步技術，指得是我們每發出一個requests都要等待server端的回應，而非同步技術可以充分利用這個等待時間，同時再發出其他requests，直到server成功回應時，才開始用同一個執行緒出處理回應回來的資料。必須注意的是，他並不會占用到多的CPU核心數，只是充分利用在同一個核心上的運算量而已。

3.異步操作無須額外的線程負擔，並且使用回調的方式進行處理，在設計良好的情況下，處理函數可以不必使用共享變量（即使無法完全不用，最起碼可以減少共享變量的數量），減少了死鎖的可能。

4.異步沒有創建其他的線程，一個線程去干其他的事情去了，那數據的讀取異步執行是去由誰完成的呢？實際上，本質是這樣的。

    熟悉電腦硬件的朋友肯定對DMA這個詞不陌生，硬盤、光驅的技術規格中都有明確DMA的模式指標，其實網卡、聲卡、顯卡也是有DMA功能的。DMA就是直接內存訪問的意思，也就是說，擁有DMA功能的硬件在和內存進行數據交換的時候可以不消耗CPU資源。只要CPU在發起數據傳輸時發送一個指令，硬件就開始自己和內存交換數據，在傳輸完成之後硬件會觸發一個中斷來通知操作完成。這些無須消耗CPU時間的I/O操作正是異步操作的硬件基礎。所以即使在DOS 這樣的單進程（而且無線程概念）系統中也同樣可以發起異步的DMA操作。

    即CPU在數據的長時間讀取過程中，只需要做兩件事，第一發布指令，開始數據交換；第二，交換結束，得到指令，CPU再進行後續操作。而中間讀取數據漫長的等待過程，CPU本身就不需要參與，順序執行就是我不參與但是我要乾等著，效率低下；異步執行就是，我不需要參與那我就去干其他事情去了，你做完了再通知我就可以了（回調）。

   但是你想一下，如果有一些異步操作必須要CPU的參與才能完成呢，即我開始的那個線程是走不開的，這該怎麼辦呢，在.NET中，有線程池去完成，線程池會高效率的開啟一個新的線程去完成異步操作，在python中這是系統自己去安排的，無需人工干預，這就比自己創建很多的線程更加高效。
   
----------------------
缺點：
1.每個請求在執行的過程中，比同步還要占用更多資源

2.編寫異步操作的複雜程度較高，程序 主要使用回調方式進行處理，與普通人的思維方式有些初入，而且難以調試。

3.“異步” ，異步最大的問題在於“回調”，這增加了軟件設計上的難度。

----------------------
多線程爬蟲
優點：
使用平行處理，同時間多個爬蟲程式重複執行
多執行緒在做的實情，無非就是，使用多個CPU核心去處理事情，這邊要比較注意的是，假設你的CPU核心不夠，硬是將執行緒的數量開的比核心數多，將可能會被強制終止某些執行緒。
多線程爬蟲連續開很多個瀏覽器，每個request各自獨立，CPU利用率高，各自獨立易理解

----------------------
缺點：
只能在單個CPU Core下執行,因為GIL(Global Interpreter Lock) 導致python 無法活用 CPU 多核的效率，僅能使用單核

所以線程需要操作系統投入CPU資源來運行和調度。
，線 程的使用（濫用）會給系統帶來上下文切換的額外負擔。並且線程間的共享變量可能造成死鎖的出現

“多線程”，第一、最大的問題在於線程本身的調度和運行需要很多時間，因此不建議自己創建太大量的線程；第二、共享資源的調度比較難，涉及到死鎖，上鎖等相關的概念。

----------------------
在實際設計時，我們可以將兩者結合起來：
*（1）當需要執行I/O操作時，使用異步操作比使用線程+同步I/O操作更合適。I/O操作不僅包括了直接的文件、網絡的讀寫，還包括數據庫操作、Web Service、HttpRequest以及.net Remoting等跨進程的調用。異步特別適用於大多數IO密集型的應用程序。
*（2）而線程的適用範圍則是那種需要長時間CPU運算的場合，例如耗時較長的圖形處理和算法執行。但是往往由於使用線程編程的簡單和符合習慣，所以很多朋友往往會使用線程來執行耗時較長的I/O操作。這樣在只有少數幾個並發操作的時候還無傷大雅，如果需要處理大量的並發操作時就不合適了。


2、在多線程編程和異步編程上的進步（以.NET和python語言為例進行說明）

多線程最大的問題在於“線程的調度”，而在.NET中引入了線程池的概念，避免人為創建多餘線程，讓系統進行分配；而Python語言中的多線程編程效率一直飽受詬病，因此python很少使用多線程，一般使用“協程”，後面會講到。

異步最大的問題在於“回調”，.NET隨著版本的升級，從多任務編程，到後面使用await-async關鍵字的提升，讓人更加方便，不用注意到復雜的回調問題，用同步的思維方式進行異步編程。python從3.4版本開始也開始使用協程和時間循環機制，從yield和yield from 語句中受到啟發，解決了“回調問題”，而在python3.5版本中，也是使用了await-async關鍵字，進一步簡化了編程。在這一方面，python和.NET有很多相似之處。（詳細案例會在後面文章中展示）

 (來源) https://www.itread01.com/content/1553169850.html